In [1]:
import pandas as pd 
import numpy as np
from pytrends.request import TrendReq
pytrend = TrendReq()
from datetime import datetime,timedelta


In [2]:
# get current date
today=datetime.today()

In [3]:
'''
since fiscal week start with Sunday and ends with Saturday, we need to get Suns and Sats for each week 
and end with most recent Sats
'''
def recent_sat_sun():
    idx = (today.weekday() + 1) % 7 
    sun = today - timedelta(7+idx)
    sat = today - timedelta(7+idx-6)
    if sat== max([sun, sat]):
        return [sat,sat-timedelta(6)]
    else:
        return [sun-timedelta(1),sun]

In [4]:
recent_weeks= recent_sat_sun()

In [5]:
'''
def recent_sat_sun():
    idx = (today.weekday() + 1) % 7 
    sun = today - timedelta(7+idx)
    sat = today - timedelta(7+idx-6)
    if sat== max([sun, sat]):
        return [sat.strftime("%m/%d/%Y"),(sat-timedelta(6)).strftime("%m/%d/%Y")]
    else:
        return [(sun-timedelta(1)).strftime("%m/%d/%Y"),sun.strftime("%m/%d/%Y")]
'''

'\ndef recent_sat_sun():\n    idx = (today.weekday() + 1) % 7 \n    sun = today - timedelta(7+idx)\n    sat = today - timedelta(7+idx-6)\n    if sat== max([sun, sat]):\n        return [sat.strftime("%m/%d/%Y"),(sat-timedelta(6)).strftime("%m/%d/%Y")]\n    else:\n        return [(sun-timedelta(1)).strftime("%m/%d/%Y"),sun.strftime("%m/%d/%Y")]\n'

In [6]:
# all Suns in 2021
start_dt=pd.date_range(start='12/27/2020',freq='7D',periods=52)
start_dt.shape

(52,)

In [7]:
# all Sats in 2021
end_dt=pd.date_range(start='1/02/2021',freq='7D',periods=52)
end_dt.shape

(52,)

In [8]:
# YTD Sats
sat_lst=[]
for i in start_dt:
    if i<=recent_weeks[1]:
        sat_lst.append(i.strftime("%m/%d/%Y"))

In [9]:
len(sat_lst)

45

In [10]:
#YTD Suns
sun_lst=[]
for i in end_dt:
    if i<=recent_weeks[0]:
        sun_lst.append(i.strftime("%m/%d/%Y"))

In [11]:
len(sun_lst)

45

In [12]:
#all weeks in 2021
dates = pd.DataFrame(start_dt, end_dt).reset_index().rename(columns={'index':'Sat',0:'Sun'})
dates['wk_nbrs']=pd.Series(list(range(202101,202153)))

In [13]:
dates.head()

,Sat,Sun,wk_nbrs
0,2021-01-02,2020-12-27,202101
1,2021-01-09,2021-01-03,202102
2,2021-01-16,2021-01-10,202103
3,2021-01-23,2021-01-17,202104
4,2021-01-30,2021-01-24,202105


In [14]:
#wk_nbrs=np.array(range(202101,202153))
#wk_nbrs.shape=(52,1)
#wk_nbrs

In [15]:
# YTD weeks in 2021
current_dates = pd.DataFrame(sat_lst,sun_lst).reset_index().rename(columns={'index':'Sat',0:'Sun'})                                                    

In [16]:
dates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
Sat        52 non-null datetime64[ns]
Sun        52 non-null datetime64[ns]
wk_nbrs    52 non-null int64
dtypes: datetime64[ns](2), int64(1)
memory usage: 1.3 KB


In [17]:
current_dates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 2 columns):
Sat    45 non-null object
Sun    45 non-null object
dtypes: object(2)
memory usage: 848.0+ bytes


In [18]:
#current_dates['Sat']=current_dates['Sat'].astype('datetime64[ns]')
#current_dates['Sun']=current_dates['Sun'].astype('datetime64[ns]')

In [19]:
# YTD weeks including week_nbrs
dates_ytd = dates.iloc[:current_dates.shape[0],:]
dates_ytd.head()

,Sat,Sun,wk_nbrs
0,2021-01-02,2020-12-27,202101
1,2021-01-09,2021-01-03,202102
2,2021-01-16,2021-01-10,202103
3,2021-01-23,2021-01-17,202104
4,2021-01-30,2021-01-24,202105


In [20]:
# build first weeks' google search trend
dt_range = dates_ytd.iloc[0,1].strftime("%Y-%m-%d")+' '+dates_ytd.iloc[0,0].strftime("%Y-%m-%d")

In [21]:
pytrend.build_payload(kw_list=['COVID Vaccine'],geo='US', timeframe=dt_range)

In [22]:
# Interest by DMA
df = pytrend.interest_by_region(resolution='DMA', inc_low_vol=True, inc_geo_code=False)\
.reset_index().sort_values('geoName')
df.head(10)

,geoName,COVID Vaccine
0,Abilene-Sweetwater TX,28
1,Albany GA,17
2,Albany-Schenectady-Troy NY,24
3,Albuquerque-Santa Fe NM,23
4,Alexandria LA,9
5,Alpena MI,0
6,Amarillo TX,28
7,Anchorage AK,23
8,Atlanta GA,20
9,Augusta GA,14


In [23]:
# create empty dataframe 
init_df=pd.DataFrame(columns=list(dates_ytd['wk_nbrs']))
init_df

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,...,202136,202137,202138,202139,202140,202141,202142,202143,202144,202145


In [24]:
# add dma names for empty dataframe
DMA_NAME=df[['geoName']].rename(columns={'geoName':'DMA_NAME'})
init_df=pd.concat([DMA_NAME,init_df])
init_df.head()

,DMA_NAME,202101,202102,202103,202104,202105,202106,202107,202108,202109,...,202136,202137,202138,202139,202140,202141,202142,202143,202144,202145
0,Abilene-Sweetwater TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albany GA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albany-Schenectady-Troy NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albuquerque-Santa Fe NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alexandria LA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
'''
Output google search trend by DMA by fiscal week
Input:
keyword: search keyword
'''
def search_by_wk_dma(keyword):
    for index, row in dates_ytd.iterrows():
        dt_range=row['Sun'].strftime("%Y-%m-%d")+' '+row['Sat'].strftime("%Y-%m-%d")
        pytrend.build_payload(kw_list=[keyword],geo='US', timeframe=dt_range)
        init_df[row['wk_nbrs']] = pytrend.interest_by_region(resolution='DMA', inc_low_vol=True, inc_geo_code=False)\
        .reset_index().sort_values('geoName')[keyword]
    return init_df
                                     

In [27]:
# google search trend by DMA by fiscal week
df_wide_format = search_by_wk_dma('COVID Vaccine')

In [28]:
df_wide_format.head()

,DMA_NAME,202101,202102,202103,202104,202105,202106,202107,202108,202109,...,202136,202137,202138,202139,202140,202141,202142,202143,202144,202145
0,Abilene-Sweetwater TX,28,45,30,14,24,29,14,5,13,...,88,66,31,7,9,45,8,21,64,30
1,Albany GA,17,17,18,12,15,22,17,9,7,...,73,33,47,6,22,33,12,16,30,15
2,Albany-Schenectady-Troy NY,24,40,77,39,57,86,99,100,60,...,63,43,36,10,28,35,47,37,65,52
3,Albuquerque-Santa Fe NM,23,41,30,21,18,28,18,16,18,...,63,32,33,9,19,36,27,18,55,24
4,Alexandria LA,9,41,23,9,25,24,20,10,11,...,72,54,77,15,0,11,0,21,39,0
